<a href="https://colab.research.google.com/github/CarlosBaez11/Carlos_Baez_C/blob/main/2_Red_neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalación de las librerías y paquetes 

Instalamos la librería de Fiftyone para la visualización de los datos, y gdown para poder descargar archivos directamente desde Drive

In [ ]:
!pip install fiftyone
!pip install gdown

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import fiftyone
import gdown
import pandas as pd

La siguiente celda no es necesario ejecutarla, porque la utilicé para almacenar la red neuronal en Drive conforme se iba entrenando

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Ahora vamos a descargar el dataset de drive

In [ ]:
gdown.download( 'https://drive.google.com/u/1/uc?id=1AT9bncPf5USGFdGj5vAv3hy_3QNSENRe&export=download', '/content/frutas.zip' )

Extraemos el contenido del archivo en zip

In [ ]:
!unzip /content/frutas.zip

Con la librería de fiftyone vamos a visualizar el contenido de nuestro dataset

In [ ]:
dataset=fiftyone.Dataset.from_dir(dataset_dir='/content/content/datasetfrutas_filtrado', label_field='ground_truth',
                                  name='Dataset_frutas11_', dataset_type=fiftyone.types.ImageClassificationDirectoryTree
                                  )

In [ ]:
fiftyone.launch_app(dataset)

Hasta el momento hemos utilizado los datasets de Fiftyone para exportar, visualizar y realizar un filtrado a las imágenes. Ahora vamos a crear los datasets de Tensorflow para entrenar a la red neuronal.



Después de probar varias veces la red neuronal con su conexión al PLC, obtuvimos un mejor rendimiento con un tamaño de imagen de 360*360

In [ ]:
IMG_SIZE=(360,360)
BATCH_SIZE=64
IMG_SHAPE=IMG_SIZE+(3,)

Ahora vamos a crear los datasets de entrenamiento y de validacion. En este caso vamos a utilizar la clase de datasets de Tensorflow, utilizando ocomo metodo de obtencion del dataset load/_image_dataset_from_directory

In [ ]:
#Vamos a crear el dataset de entrenamiento
ds_training=keras.utils.image_dataset_from_directory('/content/content/datasetfrutas_filtrado', labels='inferred',
                                                     label_mode='int', batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2, image_size=IMG_SIZE, subset='training', seed=50)

In [ ]:
ds_test=keras.utils.image_dataset_from_directory('/content/content/datasetfrutas_filtrado',
                                                 labels='inferred', label_mode='int', batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2, image_size=IMG_SIZE, subset='validation', seed=50)

Dado que esta red neronal fue puesta en producción, consideramos necesario crear un Dataframe con las clases que contiene el dataset con el que fue entrenada, para posteriormente exportarlo como un CSV y mapear el resultado de las predicciones con el nombre correspondiente a cada fruta

In [ ]:
class_names=ds_training.class_names

In [ ]:
classes=pd.DataFrame(class_names)

In [ ]:
classes.to_csv('/content/classes.csv', index=False)

Ahora vamos a visuzalizar los datos del dataset

In [ ]:
plt.figure(figsize=(10,10))
for images, labels in ds_training.take(1):
  for i in range (9):
    ax=plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])

Con el propósito de reducir timepo de entrenamiento en la medida de lo posible, y poder realizar cambios en la arquitectura rápidamente, vamos a aplicar transfer learning. Por tanto, descargamos los pesos de la arquitectura. En este caso seleccionamos la arquitectura VGG 16

In [ ]:
base_model=keras.applications.VGG19(include_top=False, input_shape=IMG_SHAPE,pooling='avg', classifier_activation=False, weights='imagenet' )

Ahora tenemos que congelar los pesos del modelos para evitar que durante la primera etapa del entrenamiento de las capas densas se pierdan las caracteristicas preentrenadas

In [ ]:
for i in base_model.layers:
  i.trainable=False

Ahora que tenemos el modelo congelado, vamos a crear la arquitectura de la red neuronal con la API funcional de Keras. Vamos a añadir algunas capas para aumentar la cantidad de los datos, después van a ser enviados al modelo preentrenado, y la salida de este modelo alimentará las capas densas de clasificación para obtener la clase correspondiente de cada imagen.




In [ ]:
keras.backend.clear_session()

In [ ]:
input=keras.Input(shape=IMG_SHAPE)


#Utilizamos Data augmentation para aumentar la cantidad de datos de entrenamiento
x=keras.layers.Resizing(IMG_SIZE[0], IMG_SIZE[1])(input)

x=keras.layers.RandomZoom(0.5,0.5)(x)

x=keras.layers.RandomContrast(1)(x)

x=keras.layers.RandomCrop(IMG_SIZE[0], IMG_SIZE[1])(x)

x=keras.layers.RandomFlip()(x)

x=keras.layers.RandomRotation(255)(x)

x=keras.applications.vgg19.preprocess_input(x)

x=base_model(x)

x=keras.layers.Flatten()(x)

x=keras.layers.Dense(128, activation='selu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.L1L2(l1=0.01, l2=0.01), use_bias=False)(x)

outputs=keras.layers.Dense(len(class_names), activation='Softmax', kernel_initializer='glorot_normal' )(x)

model=keras.Model(inputs=input, outputs=outputs)

Ahora con el método de Summary vemos algunas de las características del modelo que definimos

In [ ]:
model.summary()

Creamos dos callbacks para poder interactuar con la red neuronal durante el entrenamiento. Usamos uno de ellos, que va a guardar el progreso del entrenamiento durante cada época, pero se encuentra comentado, dado que lo estaba guardando directamente en mi carpeta de Drive, por tanto, no es necesario ejecutarlo. El otro callback se encargar de para el entrenamiento cuando está detectando overfitting en los datos de entrenamiento.

In [ ]:
# cb=keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/red4.h5', save_freq='epoch') #No es necesario ejecutar esta celda

cb1=keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

Con las configuraciones realizadas en el modelo, podemos compilarlo

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['acc'])

Ahora aplicamos el método de .fit() para entrenar el modelo. Dado que es solo la primera etapa del entrenamiento, lo vamos a entrenar como máximo durante 15 epocas

In [ ]:
model.fit(ds_training, batch_size=BATCH_SIZE, epochs=15, validation_data=ds_test, callbacks=[cb1])

Ahora vamos a aplicar un Fine tunning para mejorar el desempeño del modelo. En esta etapa entrenaremos el modelo totalmente, pero con una tasa de aprendizaje muy baja, de tal forma que se le de un ligero ajuste que permita una mejora sustancial en la capacidad de predicción.

In [ ]:
for i in model.layers:
  i.trainable=True

Verificamos que se puedan entrenar los peso del modelo observando sus atributos 

In [ ]:
model.summary()

Para el fine tunning vamos a utilizar el mismo dataset, pero cambiando los datos de entrenamiento y de testeo para evitar el overfitting

In [ ]:
#Vamos a crear el dataset de entrenamiento
ds_training=keras.utils.image_dataset_from_directory('/content/content/datasetfrutas_filtrado', labels='inferred',
                                                     label_mode='int', batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2, image_size=IMG_SIZE, subset='training', seed=40)
ds_test=keras.utils.image_dataset_from_directory('/content/content/datasetfrutas_filtrado',
                                                 labels='inferred', label_mode='int', batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2, image_size=IMG_SIZE, subset='validation', seed=40)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(ds_training, batch_size=BATCH_SIZE, epochs=20, validation_data=ds_test, callbacks=[cb1])